In [1]:
# Efficacy: https://www.hindawi.com/journals/scn/2018/5483472/#conclusions 
# Source: https://www.kaggle.com/anishpai/intro-to-credit-card-fraud-detection 
# Data Source: https://www.kaggle.com/mlg-ulb/creditcardfraud 

# Library Loads
import numpy as np
import matplotlib.pyplot as plt
import keras
import pandas as pd
data = pd.read_csv('https://sabwoody.blob.core.windows.net/backups/creditcard.csv')

# Data Descriptions
data.head()
data.dtypes
data.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,1.165980e-15,3.416908e-16,-1.373150e-15,2.086869e-15,9.604066e-16,1.490107e-15,-5.556467e-16,1.177556e-16,-2.406455e-15,...,1.656562e-16,-3.444850e-16,2.578648e-16,4.471968e-15,5.340915e-16,1.687098e-15,-3.666453e-16,-1.220404e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [26]:
# Deal with missing data
data.isnull().sum()
data.describe()

# Show percentages of + versus - in training set
print('No Fraud detected is ', round(data['Class'].value_counts()[0]/len(data) * 100,2), '% of the dataset')
print('Fraud detected is  ', round(data['Class'].value_counts()[1]/len(data) * 100,2), '% of the dataset')
print('(Fraudulent Transactions: ', data['Class'].value_counts()[1], ' out of ', data['Class'].value_counts()[0], ' records)')

No Fraud detected is  99.83 % of the dataset
Fraud detected is   0.17 % of the dataset
(Fraudulent Transactions:  492  out of  284315  records)


In [27]:
# Machine Learning approach - data prep

## Let's deal with outliers first
from sklearn.preprocessing import StandardScaler, RobustScaler
std_scaler = StandardScaler()
rob_scaler = RobustScaler()
data['scaled_amount'] = rob_scaler.fit_transform(data['Amount'].values.reshape(-1,1))
data['scaled_time'] = rob_scaler.fit_transform(data['Time'].values.reshape(-1,1))

In [28]:
## More data cleaning, prep for graphics if desired
data.drop(['Time','Amount'], axis=1, inplace=True)
amount = data['scaled_amount']
time = data['scaled_time']
data.drop(['scaled_amount', 'scaled_time'], axis=1, inplace=True)
data.insert(0, 'amount', amount)
data.insert(1, 'time', time)

# Examine data cleaned so far:
data.head()

,amount,time,V1,V2,V3,V4,V5,V6,V7,V8,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Class
0,1.783274,-0.994983,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,...,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0
1,-0.269825,-0.994983,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,...,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,0
2,4.983721,-0.994972,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,...,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,0
3,1.418291,-0.994972,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,...,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0
4,0.670579,-0.994960,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,...,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,0


In [30]:
data = data.sample(frac=1)

fraud_data = data.loc[data['Class']==1]
nfraud_data = data.loc[data['Class']==0][:492]

normal_distributed_df = pd.concat([fraud_data, nfraud_data])

# Shuffle dataframe rows
ndata = normal_distributed_df.sample(frac=1, random_state=42)
ndata.head()

print('Fraudulent Transactions', data['Class'].value_counts()[1])
print('Non-Fraudulent Transactions', data['Class'].value_counts()[0])

Fraudulent Transactions 492
Non-Fraudulent Transactions 284315


In [31]:
X= data.iloc[:, data.columns != 'Class']
y = data.iloc[:, data.columns == 'Class']

dataframe = pd.DataFrame(data=ndata)

dataframe

X= dataframe.iloc[:, ndata.columns != 'Class']
y = dataframe.iloc[:, ndata.columns == 'Class']

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.25, random_state=1)

X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)



In [32]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'gini', random_state= 0 )
classifier.fit(X_train, y_train.ravel())
from sklearn.metrics import classification_report, confusion_matrix

cm_grid = confusion_matrix(y_test,y_pre)
y_pre = classifier.predict(X_test)
print(classification_report(y_test,y_pre))

classifier.score(X_test,y_test)


              precision    recall  f1-score   support

           0       0.90      0.93      0.91       121
           1       0.93      0.90      0.91       125

    accuracy                           0.91       246
   macro avg       0.91      0.91      0.91       246
weighted avg       0.91      0.91      0.91       246



0.9146341463414634

In [33]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout


#Initializing ANN
classifier = Sequential()

#Input Layer
classifier.add(Dense(30, activation='relu'))
#2nd layer
classifier.add(Dense(16, activation='relu'))
classifier.add(Dense(16, activation='relu'))

#Output layer
classifier.add(Dense(1, activation='sigmoid'))

#Compling the ANN
classifier.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])

#Fitting the dataset into ANN
classifier.fit(X_train, y_train, batch_size=100, epochs=100, verbose = 0)

score = classifier.evaluate(X_test, y_test, verbose = 0) 
print('Test loss:', score[0]) 
print('Test accuracy:', score[1])

#Predicting the results
y_pred = classifier.predict(X_test)


Test loss: 0.33069172501564026
Test accuracy: 0.9268292784690857
